# Prerequisites

If running locally, here is the list of modules that you need: jupyter, matplotlib, numpy, pandas, scipy, scikit-learn

One way to install them is running this command:
```
python3 -m pip install -U jupyter matplotlib numpy pandas scipy scikit-learn
```

The flag "U" upgrades all specified packages to the newest.

In VScode, you can install the extension "Code Spell Checker" to help you avoid typos in your markdown cells.

# Introduction

We want to use California census data to build a Linear Regression model of housing prices in the state. This data includes several metrics for each district (a district typically has a population of 600 to 3000 people). The model should learn from this data and be able to predict the median housing price in any district, given all the other metrics. The data that we use in this practice is not recent, but no harm in pretending it so.

What is your target? Median housing price.

Is this task going to be a supervised learning or otherwise? Supervised.

# Load data and take a sneak peek

In [ ]:
#
import pandas as pd
housing

Now, you are seeing the different attributes that are present in the data. Each row represents one district in California. Now, let's use the ```info()``` method to get a brief description of the data:

In [ ]:
#
housing

We can now see information such as total number of rows, attributes' types, and number of nonnull values. Notice that the "total_bedrooms" attribute has 20433 nonnull values while we see a total of 20640 data entries. This means that 207 (20640-20433=207) rows (i.e., districts) are missing this feature. We will take care of these rows later.

We also notice that all attributes are numerical, except "ocean_proximity" that is "object", so it could hold any kind of Python object. When you used `head()` method, you probably noticed that the values in the "ocean_proximity" column were repetitive, which means that it is probably a categorical attribute. Now, let's figure out what categories exist and how many districts belong to each category by using the `value_counts()` method:

In [ ]:
#
housing

We can also use the `describe()` method to get a statistical summary of the numerical attributes:

In [ ]:
#
housing

The rows "count", "mean", "min", and "max" are self-explanatory. Note that the method `describe` ignores null values and that's why the "count" of "total_bedrooms" is 20,433, and not 20,640). "std" row shows the standard deviation, which measures how dispersed the values are. The 25%, 50%, and 75% rows show the corresponding percentiles.

Recap: A percentile indicates the value below which a given percentage of observations in a group of observations fall. For example, 25% of the districts have a "housing_median_age" lower than 18, while 50% are lower than 29, and 75% are lower than 37. These are often called the 25th percentile (or first quartile), the median, and the 75th percentile (or third quartile).

Another quick way to get a feel of the data you are dealing with is to plot a histogram for each numerical attribute.

Recap: A histogram shows the number of instances (on the vertical axis) that have a given value range (on the horizontal axis). You can either plot one attribute at a time, or you can call the `hist()` method on the whole dataset and it will plot a histogram for each numerical attribute:

In [ ]:
#
import matplotlib.pyplot as plt
housing

Now you should carefully investigate every histogram tile and try to derive insights. A few example insights:
- The median income attribute does not look like it is expressed in dollars. You inquire into it and you are told that the data has been scaled and capped at 15 for higher median incomes, and at 0.5 for lower median incomes. For example, 3 actually means about $30,000. So, in this case, you are dealing with a preprocessed attribute. Data preprocessing is a common practice in data science and not necessarily a problem. But you should try to catch things like this, inquire into them, and make sure that your understanding of the data is correct.

- You are also told the housing median age and the median house value were also capped. The latter may be a serious problem since it is your target attribute. Your ML algorithm may learn that prices never go beyond that limit. You need to check with the team that will use your model’s output to see if this is a problem or not. If they tell you that they need precise predictions even beyond $500000, then you have two options:

  1. Collect proper labels for the districts whose labels were capped.

  2. Remove those districts from the date. Note that eliminating those does not count as *dimensionality reduction*, because you are dropping data points - and not features.

- The attributes have very different scales suggesting feature scaling.
- Many of the histograms extend much farther to the right of the median than to the left. This may make it harder for your ML algorithms to detect patterns. Perhaps we should consider transforming these attributes to have more bell-shaped distributions.

# Split the data into training and test set

Pick randomly 20% (or less if your dataset is very large) of the dataset instances and set them aside. This subset will be your test set and the rest will be your training set.

Note that we set the random number generator’s seed before running the splitter, so that it always generates the same shuffled indices.

In [ ]:
#
from sklearn.model_selection import train_test_split
seed = 2835
train_set, test_set = 

We did a purely random sampling. This is generally fine if your dataset is large enough (especially relative to the number of attributes), but if it is not, you run the risk of introducing a significant sampling bias. When a survey company decides to call 1000 people to ask them a few questions, they don’t just pick 1000 people randomly in a phone book. They try to ensure that these 1000 people are representative of the whole population. For example, the US population is 51.3% females and 48.7% males, so a well-conducted survey in the US would try to maintain this ratio in the sample: 513 female and 487 male. This is called __stratified sampling__: the population is divided into homogeneous subgroups called __strata__, and the right number of instances are sampled from each __stratum__ to guarantee that the test set is representative of the overall population. If the people running the survey used purely random sampling, there would be about a 12% chance of sampling a skewed test set that was either less than 49% female or more than 54% female. Either way, the survey results would be significantly biased.

Suppose you chatted with experts who told you that the median income is a very important attribute to predict median housing prices. You may want to ensure that the test set is representative of the various categories of incomes in the whole dataset. Since the median income is a continuous numerical attribute, you first need to create an income category attribute. Let’s look at the median income histogram more closely:

In [ ]:
#
housing

We observe that most median income values are clustered around 1.5 to 6 (i.e., $15000–$60000), but some median incomes go far beyond 6. It is important to have a sufficient number of instances in your dataset for each stratum, otherwise the estimate of a stratum’s importance may be biased. This means that you should not have too many strata, and each stratum should be large enough. Let's use the `pd.cut()` function to create an income category attribute with five categories (labeled from 1 to 5): category 1 ranges from 0 to 1.5 (i.e., less than $15000), category 2 from 1.5 to 3, and so on:

In [ ]:
#
import numpy as np
housing["income_cat"] =

In [ ]:
#
housing["income_cat"].hist()
plt.show()

We are now ready to do stratified sampling based on the income category that we just created. For this we can use Scikit-Learn’s `StratifiedShuffleSplit` class:

In [ ]:
#
from sklearn.model_selection import StratifiedShuffleSplit

split = 

Let’s see if this worked as expected. You can start by looking at the income category proportions in the test set:

In [ ]:
#
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

Let's compare the income category proportions in the full dataset, in the test set generated with stratified sampling, and in a test set generated using purely random sampling.

In [ ]:
#
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=seed)

compare_props = pd.DataFrame({
    "Full": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()

compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Full"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Full"] - 100

compare_props

As you can see, the test set generated using stratified sampling has income category proportions almost identical to those in the full dataset, whereas the test set generated using purely random sampling is skewed.

Now let's remove the "income_cat" attribute so the data is back to its original state:

In [ ]:
#
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Gain more insights

Let’s create a copy of our stratified training set so that we can play with it without harm:

In [ ]:
#
housing = strat_train_set.copy()

Since the data includes geographical information (latitude and longitude), it is a good idea to visualize the data for all districts using scatter plot:

In [ ]:
#
housing.
plt.show()

But it is hard to see any particular pattern. Setting the alpha option to 0.1 makes it much easier to visualize the places where there is a high density of data points:

In [ ]:
#
housing.
plt.show()

Now you can clearly see the high-density areas, like the Bay Area. Our brains are very good at spotting patterns in pictures, but you may need to play around with visualization parameters to make the patterns stand out or more pronounced.

Now let’s look at the housing prices. The radius of each circle represents the district’s population (option `s`), and the color represents the price (option `c`). We will use a predefined color map (option `cmap`) called jet , which ranges from blue (low values) to red (high prices):

In [ ]:
#
housing

plt.legend()
plt.show()

This image tells you that the housing prices are very much related to the location (for example close to the ocean) and to the population density, as you probably knew already. A clustering algorithm should be useful for detecting the main cluster and for adding new features that measure the proximity to the cluster centers (*i.e.*, example of feature extraction using unsupervised techniques).

# Let's find correlations

We can compute the Pearson’s Correlation Coefficient (PCC, or Pearson’s r) to measure linear correlation measure between every pair of attributes using the `corr()` method; and see how much each attribute correlates with the median house value (*i.e.*, target):

In [ ]:
#
corr_matrix =

The correlation coefficient ranges from –1 to 1. When it is close to 1, it means that there is a strong positive correlation; for example, the median house value tends to go up when the median income goes up. When the coefficient is close to –1, it means that there is a strong negative correlation; you can see a small negative correlation between the latitude and the median house value (i.e., prices have a slight tendency to go down when you go north). Finally, coefficients close to 0 mean that there is no **linear** correlation. The correlation coefficient only measures linear correlations and it may completely miss out on nonlinear relationships.

There are different methods to measure nonlinear correlation, one of them being [MIC](https://en.wikipedia.org/wiki/Maximal_information_coefficient).

Another way to check for correlation between attributes is to use the pandas `scatter_matrix()` function, which plots every numerical attribute against every other numerical attribute. Since there are now 11 numerical attributes, we would get 11*11 = 121 plots, which would not fit on a page. So, let’s just focus on a few promising
attributes that seem most correlated with the median housing value:

In [ ]:
#
from pandas.plotting import scatter_matrix


Note that the main diagonal (top left to bottom right) does not show correlation tiles, instead it is where pandas displays a histogram of each attribute. We saw in the correlation matrix also in these correlation tiles that the most promising attribute to predict the median house value is the median income, so let’s zoom in on their correlation scatter plot:

In [ ]:
#
housing.
plt.show()

This plot reveals a few things. First, the correlation is indeed very strong; you can clearly see the upward trend, and the points are not too dispersed. Second, the price cap that we noticed earlier is clearly visible as a horizontal line at $500000. But this plot reveals other less obvious straight lines: a horizontal line around $450000, another around $350000, perhaps one around $280000, and a few more below that. You may want to try removing the corresponding districts to prevent your algorithms from learning to reproduce these data quirks.

# Experiment with attribute combinations

One last thing you may want to do before preparing the data for your ML is to try out various attribute combinations. For example, the total number of rooms in a district is not very useful if you don’t know how many households there are. What you really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful: you probably want to compare it to the number of rooms. And the population per household also seems like an interesting attribute combination to look at. Let’s create these new attributes:

In [ ]:
#
housing["rooms_per_household"]      =
housing["bedrooms_per_room"]        = 
housing["population_per_household"] = 

And now let’s look at the correlation matrix again:

In [ ]:
#


The new "bedrooms_per_room" attribute is much more correlated with the median house value than the total number of rooms or bedrooms. The number of rooms per household is also more informative than the total number of rooms in a district.

# Prepare the data for ML

First let’s revert to a clean training set (by copying "strat_train_set" once again). Let’s also separate the predictors and the labels, since we don’t necessarily want to apply the same transformations to the predictors and the target values:

In [ ]:
#
housing = 
housing_labels = 

# Data cleaning

Most Machine Learning algorithms cannot work with missing features, so let’s take care of them. We saw earlier that the "total_bedrooms" attribute has some missing values, so let’s fix this. We have three options:
   - Get rid of the corresponding districts
   - Get rid of the whole attribute
   - Set the values to some value (zero, the mean, the median, etc.).

You can accomplish these easily using DataFrame’s `dropna()` , `drop()` , and `fillna()` methods:

In [ ]:
# option 1:
# housing.dropna(subset=["total_bedrooms"])

# option 2:
# housing.drop("total_bedrooms", axis=1)

# option 3:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True)

Scikit-Learn also provides a handy class to take care of missing values: `SimpleImputer`. First, you need to create a `SimpleImputer` instance, specifying that you want to replace each attribute’s missing values with the median of that attribute. Since the median can only be computed on numerical attributes, you need to create a
copy of the data without the text attribute "ocean_proximity". Then, you can fit the imputer instance to the training data using the `fit()` method:

In [ ]:
#
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

Note that any object that can estimate some parameters based on a dataset is called an estimator (e.g., an imputer is an estimator). All the estimator’s hyperparameters are accessible directly via public instance variables (e.g., imputer.strategy), and all the estimator’s learned parameters are accessible via public instance variables with an underscore suffix (e.g., imputer.statistics_).

In [ ]:
#
print(imputer.strategy)
imputer.statistics_

In [ ]:
#
housing_num.median().values

Now you can use this "trained" imputer to transform the training set by replacing missing values with the learned medians and the result is a plain NumPy array containing the transformed features:

In [ ]:
#
X = imputer.transform(housing_num)

And, you can put it back into a pandas DataFrame:

In [ ]:
#
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)
housing_tr.head()

# Handling text and categorical attributes

In this dataset, there is only one text attribute that is the "ocean_proximity". Let’s look at its value for the first 10 instances:

In [ ]:
#
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(n=10)

It’s not arbitrary text and there are a limited number of possible values, each of which represents a category. So this attribute is a categorical attribute. Most Machine Learning algorithms prefer to work with numbers, so let’s convert these categories from text to numbers (i.e. encoding). For this, we can use Scikit-Learn’s `OrdinalEncoder` class:

In [ ]:
#
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = 

You can get the list of categories using the `categories_` instance variable. It is a list containing a 1D array of categories for each categorical attribute (in this case, a list containing a single array since there is just one categorical attribute):

In [ ]:
#
ordinal_encoder.categories_

One issue with this representation is that ML algorithms will assume that two nearby values are more similar than two distant values. This may be fine in some cases (e.g., for ordered categories such as "bad," "average," "good," and "excellent"), but it is obviously not the case for the "ocean_proximity" column (for example, categories 0 and 4 are clearly more similar than categories 0 and 1). To fix this issue, a common solution is to create one binary attribute per category: one attribute equal to 1 when the category is "<1H OCEAN" (and 0 otherwise), another attribute equal to 1 when the category is "INLAND" (and 0 otherwise), and so on. This is called *one-hot* encoding, because only one attribute will be equal to 1 (hot), while the others will be 0 (cold). The new attributes are sometimes called dummy attributes. Scikit-Learn provides a `OneHotEncoder` class to convert categorical values into one-hot vectors:

In [ ]:
#
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = 

Notice that the output is a SciPy sparse matrix, instead of a NumPy array. Sparse matrix only stores the location of the nonzero elements, saving memory and computation. However, you can convert it to a NumPy array if you wish:

In [ ]:
#
housing_cat_1hot.toarray()

And again, you can get the list of categories using the encoder’s `categories_` instance variable:

In [ ]:
#
cat_encoder.categories_

# Transformation

Some estimators (such as an imputer) can also transform a dataset; these are called transformers. The transformation is performed by the `transform()` method with the dataset to transform as a parameter. It returns the transformed dataset. This transformation generally relies on the learned parameters, as is the case for an imputer. All transformers also have a convenience method called `fit_transform()` that is equivalent to calling `fit()` and then `transform()` (but sometimes
`fit_transform()` is optimized and runs much faster).

One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don’t perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median
incomes only range from 0 to 15.

There are two common ways to get all attributes to have the same scale: *min-max* scaling and *standardization*.

- Min-max scaling (many people call this normalization) is the simplest: values are shifted and rescaled so that they end up ranging from 0 to 1. We do this by subtracting the min value and dividing by the max minus the min. Scikit-Learn provides a transformer called `MinMaxScaler` for this. It has a `feature_range` hyperparameter that lets you change the range if, for some reason, you don’t want 0–1.
  
- Standardization is different: first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the standard deviation so that the resulting distribution has unit variance. Unlike min-max scaling, standardization does not bound values to a specific range, which may be a problem for some algorithms (e.g., neural networks often expect an input value ranging from 0 to 1). However, standardization is much less affected by outliers. For example, suppose a district had a median income equal to 100 (by mistake). Min-max scaling would then crush all the other values from 0–15 down to 0–0.15, whereas standardization would not be much affected. Scikit-Learn provides a transformer called `StandardScaler` for standardization.

Fortunately, Scikit-Learn provides the `Pipeline` class to help you execute many data transformation steps in the right order. However, for tasks such as custom cleanup operations or combining specific attributes, you will need to write your own transformer. You will want your custom transformer to work seamlessly with Scikit-Learn functionalities (such as `Pipelines`), and since Scikit-Learn relies on duck typing (not inheritance), all you need to do is create a class and implement three methods: `fit()` (returning self), `transform()`, and `fit_transform()`.

You can get the last one for free by simply adding `TransformerMixin` as a base class. If you add `BaseEstimator` as a base class, you will also get two extra methods (`get_params()` and `set_params()`) that will be useful for automatic hyperparameter tuning.

For example, here is a small transformer class that adds the combined attributes we discussed earlier:

In [ ]:
#
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

housing_extra_attribs

We just saw that "housing_extra_attribs" is a NumPy array, we've lost the column names (unfortunately, that's a problem with Scikit-Learn). To recover a dataFrame, we run:

In [ ]:
#
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)

housing_extra_attribs.head()

We can now use Scikit-Learn's `Pipeline` to group our custom transformer along with the standard ones (i.e. the scalers) to be executed in the right order:

In [ ]:
#
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
#
housing_num_tr

We've handled the categorical columns and the numerical columns separately. It would be more convenient to have a single transformer able to handle all columns, applying the appropriate transformations to each column. Scikit-Learn has the `ColumnTransformer` for this purpose that works great with pandas DataFrames. Let’s use it to apply all the transformations to the housing data:

In [ ]:
#
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

# Note that we construct a ColumnTransformer. The constructor requires a list of tuples, where each tuple contains a
# name, a transformer, and a list of names (or indices) of columns that the transformer should be applied to.
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

# Finally, we apply this ColumnTransformer to the housing data: it applies each transformer to the appropriate columns
# and concatenates the outputs along the second axis (the transformers must return the same number of rows).
housing_prepared = full_pipeline.fit_transform(housing)

housing_prepared

In [ ]:
#
housing_prepared.shape

Note that the `OneHotEncoder` returns a sparse matrix, while the `num_pipeline` returns a dense matrix. When there is such a mix of sparse and dense matrices, the `ColumnTransformer` estimates the density of the final matrix and it returns a sparse matrix if the density is lower than a given threshold (by default, `sparse_threshold=0.3`). In this example, it returns a dense matrix. And that’s it! We have a preprocessing pipeline that takes the full housing data and applies
the appropriate transformations to each column.

# Select and train a model
We are now ready to select and train a Machine Learning model. 
Thanks to all the previous steps, things are now going to be much simpler than you might think. Now, let’s train a Linear Regression model.

In [ ]:
#


You now have a working Linear Regression model. Let’s try it out on a few instances from the training set. First extract the instances:

In [ ]:
#


and try the model on the instances:

In [ ]:
#


and compare against the actual values:

In [ ]:
#


It works, though the predictions are not accurate. Let’s the RMSE of our model on the whole training set using Scikit-Learn’s `mean_squared_error()` function:

In [ ]:
#
